# QML4EO-tutorial: Hybrid Quantum Convolutional Neural Network Classifier
## This is a qiskit adapted version of https://github.com/alessandrosebastianelli/QML4EO-tutorial/tree/main


Please refer to the following articles to get more insight about this topic:

- Sebastianelli, A., Zaidenberg, D. A., Spiller, D., Le Saux, B., & Ullo, S. L. (2021). On circuit-based hybrid quantum neural networks for remote sensing imagery classification. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 15, 565-580.
- Zaidenberg, D. A., Sebastianelli, A., Spiller, D., Le Saux, B., & Ullo, S. L. (2021, July). Advantages and bottlenecks of quantum machine learning for remote sensing. In 2021 IEEE International Geoscience and Remote Sensing Symposium IGARSS (pp. 5680-5683). IEEE.
- Eurosat: A novel dataset and deep learning benchmark for land use and land cover classification. Patrick Helber, Benjamin Bischke, Andreas Dengel, Damian Borth. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 2019.
- Introducing EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification. Patrick Helber, Benjamin Bischke, Andreas Dengel. 2018 IEEE International Geoscience and Remote Sensing Symposium, 2018.
- https://qiskit.org/documentation/machine-learning/tutorials/index.html
- https://pennylane.ai/qml/demos_qml.html

# 1. Clone the base code

Basic functions are wrapped in seprated files to keep the notebook clean. Feel free to explore the code.

In [ ]:
import sys
!rm -r /content/qml4eo_tutorial_qiskit
!git clone https://github.com/alessandrosebastianelli/qml4eo_tutorial_qiskit.git
if '/content/qml4eo_tutorial_qiskit' not in sys.path: sys.path.append('/content/qml4eo_tutorial_qiskit')

# 2. Install missing requirements

AI-related

In [ ]:
!pip install -r /content/qml4eo_tutorial_qiskit/requirements.txt -q

EO-related

In [ ]:
!pip install --upgrade pyosv

QC-related

In [ ]:
!pip install -q 'qiskit>=2.0.0'
!pip install -q 'qiskit-machine-learning>=0.7.0'

# 3. Download the dataset

In this study, we address the challenge of land use and land cover classification using Sentinel-2 satellite images. The Sentinel-2 satellite images are openly and freely accessible provided in the Earth observation program Copernicus. We present a novel dataset based on Sentinel-2 satellite images covering 13 spectral bands and consisting out of 10 classes with in total 27,000 labeled and geo-referenced images. We provide benchmarks for this novel dataset with its spectral bands using state-of-the-art deep Convolutional Neural Network (CNNs). With the proposed novel dataset, we achieved an overall classification accuracy of 98.57%. The resulting classification system opens a gate towards a number of Earth observation applications. We demonstrate how this classification system can be used for detecting land use and land cover changes and how it can assist in improving geographical maps.

Get more information [here](https://github.com/phelber/EuroSAT).

![](https://github.com/phelber/EuroSAT/blob/master/eurosat_overview_small.jpg?raw=true)

The following snippet will download the RGB version of EuroSAT and slit it in train/validation/test sub-datasets.

In [ ]:
!bash /content/qml4eo_tutorial_qiskit/dowload_data.sh